In [1]:
from langchain_text_splitters.base import TextSplitter
from dotenv import load_dotenv
from typing import List, Any
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import re
import tiktoken

In [2]:
from rank_bm25 import BM25Okapi

In [3]:
from langchain_community.retrievers import BM25Retriever
from langchain_community.document_loaders import DataFrameLoader

In [4]:
import pandas as pd
import numpy as np
import os
import re
import pickle

In [34]:
import config
import importlib
import models
importlib.reload(config)
importlib.reload(models)

<module 'models' from 's:\\PLP-Project\\models.py'>

In [52]:
from qdrant_client.models import Distance, VectorParams

In [10]:
df = pd.read_csv("data/Chunked_Data.csv")

In [11]:
df

,TEXT,CHAPTER
0,The election of PresidentFranklin Delano Roose...,"Franklin Roosevelt and the New Deal, 1932-1941"
1,"Roosevelt understood the need to ""save the pat...","Franklin Roosevelt and the New Deal, 1932-1941"
2,Franklin Roosevelt was part of the political e...,"Franklin Roosevelt and the New Deal, 1932-1941"
3,As Hoover grew more morose and physically unwe...,"Franklin Roosevelt and the New Deal, 1932-1941"
4,"By the 1932 presidential election, Hoover's po...","Franklin Roosevelt and the New Deal, 1932-1941"
...,...,...
2012,"In their place came big business, with the inf...","Go West Young Man! Westward Expansion, 1840-1900"
2013,Settlers encroaching on Native American land c...,"Go West Young Man! Westward Expansion, 1840-1900"
2014,Although the Americanization policy formulated...,"Go West Young Man! Westward Expansion, 1840-1900"
2015,"In the nineteenth century, the Hispanic, Chine...","Go West Young Man! Westward Expansion, 1840-1900"


In [12]:
doc_loader = DataFrameLoader(df, page_content_column="TEXT")

In [13]:
docs = doc_loader.load()

In [15]:
bm25_retriever = BM25Retriever.from_documents(docs)

In [18]:
with open("HistoryBook.bm25","wb") as file:
    pickle.dump(bm25_retriever,file)

In [20]:
with open("HistoryBook.bm25","rb") as file:
    bm25_retriever = pickle.load(file)

In [21]:
bm25_retriever.get_relevant_documents("Franklin Roosevelt")

c:\Users\Karthik Arun\.conda\envs\EdTechQAGeneration\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content="Franklin Roosevelt was part of the political establishment and the wealthy elite, but in the 1932 presidential campaign, he did not want to be perceived that way. Roosevelt felt that the country needed sweeping change, and he ran a campaign intended to convince the American people that he could deliver that change. It was not the specifics of his campaign promises that were different; in fact, he gave very few details and likely did not yet have a clear idea of how he would raise the country out of the Great Depression. But he campaigned tirelessly, talking to thousands of people, appearing at his party's national convention, and striving to show the public that he was a different breed of politician.", metadata={'CHAPTER': 'Franklin Roosevelt and the New Deal, 1932-1941'}),
 Document(page_content="On assuming the office of president upon the death of Franklin Roosevelt, Harry Truman was already troubled by Soviet actions in Europe. He disliked the concessions m

In [30]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

In [31]:
sentences = ['That is a happy person', 'That is a very happy person']

model = SentenceTransformer('Alibaba-NLP/gte-large-en-v1.5', trust_remote_code=True)
embeddings = model.encode(sentences)

c:\Users\Karthik Arun\.conda\envs\EdTechQAGeneration\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\Karthik Arun\.conda\envs\EdTechQAGeneration\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [32]:
type(embeddings)

numpy.ndarray

In [28]:
encode = lambda sentence, model: model.encode(sentence)

In [ ]:
df['embeddings'] = df['sentences'].apply(encode, args=(model,))

In [29]:
def create_combined_text(row):

  return f"{row['CHAPTER']} {row['TEXT']}"

df['combined_text'] = df.apply(create_combined_text, axis=1)

def encode_text(text, model):

  embedding = model.encode(text)
  return embedding

In [35]:
df['combined_text'] = df['CHAPTER'] + "  " + df['TEXT']

In [36]:
df

,TEXT,CHAPTER,combined_text,embeddings
0,The election of PresidentFranklin Delano Roose...,"Franklin Roosevelt and the New Deal, 1932-1941","Franklin Roosevelt and the New Deal, 1932-1941...",[-0.75826144 -0.5804633 -1.4329212 ... 0.87...
1,"Roosevelt understood the need to ""save the pat...","Franklin Roosevelt and the New Deal, 1932-1941","Franklin Roosevelt and the New Deal, 1932-1941...",[-0.73815656 -0.7213167 -1.4992899 ... 0.65...
2,Franklin Roosevelt was part of the political e...,"Franklin Roosevelt and the New Deal, 1932-1941","Franklin Roosevelt and the New Deal, 1932-1941...",[-0.485235 -0.8373444 -1.5882397 ... 0.59...
3,As Hoover grew more morose and physically unwe...,"Franklin Roosevelt and the New Deal, 1932-1941","Franklin Roosevelt and the New Deal, 1932-1941...",[-0.71489537 -0.57621986 -1.3793063 ... 0.54...
4,"By the 1932 presidential election, Hoover's po...","Franklin Roosevelt and the New Deal, 1932-1941","Franklin Roosevelt and the New Deal, 1932-1941...",[-0.49304003 -0.5477071 -0.84136444 ... 0.41...
...,...,...,...,...
2012,"In their place came big business, with the inf...","Go West Young Man! Westward Expansion, 1840-1900","Go West Young Man! Westward Expansion, 1840-19...",[-0.09806645 -0.6857236 -0.5511301 ... 0.50...
2013,Settlers encroaching on Native American land c...,"Go West Young Man! Westward Expansion, 1840-1900","Go West Young Man! Westward Expansion, 1840-19...",[-0.36336237 -1.075941 0.50755715 ... 0.35...
2014,Although the Americanization policy formulated...,"Go West Young Man! Westward Expansion, 1840-1900","Go West Young Man! Westward Expansion, 1840-19...",[-0.63523734 -0.7510578 0.2216021 ... -0.10...
2015,"In the nineteenth century, the Hispanic, Chine...","Go West Young Man! Westward Expansion, 1840-1900","Go West Young Man! Westward Expansion, 1840-19...",[-0.2710217 -1.2888566 -0.5610585 ... 0.75...


In [39]:
if 'embeddings_new' not in df.columns:
  df['embeddings_new'] = None

In [40]:
df

,TEXT,CHAPTER,combined_text,embeddings,embeddings_new
0,The election of PresidentFranklin Delano Roose...,"Franklin Roosevelt and the New Deal, 1932-1941","Franklin Roosevelt and the New Deal, 1932-1941...",[-0.75826144 -0.5804633 -1.4329212 ... 0.87...,None
1,"Roosevelt understood the need to ""save the pat...","Franklin Roosevelt and the New Deal, 1932-1941","Franklin Roosevelt and the New Deal, 1932-1941...",[-0.73815656 -0.7213167 -1.4992899 ... 0.65...,None
2,Franklin Roosevelt was part of the political e...,"Franklin Roosevelt and the New Deal, 1932-1941","Franklin Roosevelt and the New Deal, 1932-1941...",[-0.485235 -0.8373444 -1.5882397 ... 0.59...,None
3,As Hoover grew more morose and physically unwe...,"Franklin Roosevelt and the New Deal, 1932-1941","Franklin Roosevelt and the New Deal, 1932-1941...",[-0.71489537 -0.57621986 -1.3793063 ... 0.54...,None
4,"By the 1932 presidential election, Hoover's po...","Franklin Roosevelt and the New Deal, 1932-1941","Franklin Roosevelt and the New Deal, 1932-1941...",[-0.49304003 -0.5477071 -0.84136444 ... 0.41...,None
...,...,...,...,...,...
2012,"In their place came big business, with the inf...","Go West Young Man! Westward Expansion, 1840-1900","Go West Young Man! Westward Expansion, 1840-19...",[-0.09806645 -0.6857236 -0.5511301 ... 0.50...,None
2013,Settlers encroaching on Native American land c...,"Go West Young Man! Westward Expansion, 1840-1900","Go West Young Man! Westward Expansion, 1840-19...",[-0.36336237 -1.075941 0.50755715 ... 0.35...,None
2014,Although the Americanization policy formulated...,"Go West Young Man! Westward Expansion, 1840-1900","Go West Young Man! Westward Expansion, 1840-19...",[-0.63523734 -0.7510578 0.2216021 ... -0.10...,None
2015,"In the nineteenth century, the Hispanic, Chine...","Go West Young Man! Westward Expansion, 1840-1900","Go West Young Man! Westward Expansion, 1840-19...",[-0.2710217 -1.2888566 -0.5610585 ... 0.75...,None


In [41]:
for idx, row in df.iterrows():
  text = row['combined_text']
  embedding = model.encode(text)
  df.at[idx, 'embeddings_new'] = embedding

In [30]:
df['embeddings'] = df['combined_text'].apply(encode_text, args=(model,))

In [35]:
df.to_csv("data/Embedded.csv",index=False)

In [36]:
from qdrant_manager import QdrantManager
from qdrant_client.http.models import PointStruct

In [8]:
URL = "http://localhost"
QDRANT_CONFIG = {
    "QDRANT_URL": URL,
    "QDRANT_PORT":6333
}
#qdrant_manager = QdrantManager(QDRANT_CONFIG)
#client = qdrant_manager.client

In [33]:
df = pd.read_csv("data/Embedded.csv")

In [34]:
df

,TEXT,CHAPTER,combined_text,embeddings
0,The election of PresidentFranklin Delano Roose...,"Franklin Roosevelt and the New Deal, 1932-1941","Franklin Roosevelt and the New Deal, 1932-1941...",[-0.75826144 -0.5804633 -1.4329212 ... 0.87...
1,"Roosevelt understood the need to ""save the pat...","Franklin Roosevelt and the New Deal, 1932-1941","Franklin Roosevelt and the New Deal, 1932-1941...",[-0.73815656 -0.7213167 -1.4992899 ... 0.65...
2,Franklin Roosevelt was part of the political e...,"Franklin Roosevelt and the New Deal, 1932-1941","Franklin Roosevelt and the New Deal, 1932-1941...",[-0.485235 -0.8373444 -1.5882397 ... 0.59...
3,As Hoover grew more morose and physically unwe...,"Franklin Roosevelt and the New Deal, 1932-1941","Franklin Roosevelt and the New Deal, 1932-1941...",[-0.71489537 -0.57621986 -1.3793063 ... 0.54...
4,"By the 1932 presidential election, Hoover's po...","Franklin Roosevelt and the New Deal, 1932-1941","Franklin Roosevelt and the New Deal, 1932-1941...",[-0.49304003 -0.5477071 -0.84136444 ... 0.41...
...,...,...,...,...
2012,"In their place came big business, with the inf...","Go West Young Man! Westward Expansion, 1840-1900","Go West Young Man! Westward Expansion, 1840-19...",[-0.09806645 -0.6857236 -0.5511301 ... 0.50...
2013,Settlers encroaching on Native American land c...,"Go West Young Man! Westward Expansion, 1840-1900","Go West Young Man! Westward Expansion, 1840-19...",[-0.36336237 -1.075941 0.50755715 ... 0.35...
2014,Although the Americanization policy formulated...,"Go West Young Man! Westward Expansion, 1840-1900","Go West Young Man! Westward Expansion, 1840-19...",[-0.63523734 -0.7510578 0.2216021 ... -0.10...
2015,"In the nineteenth century, the Hispanic, Chine...","Go West Young Man! Westward Expansion, 1840-1900","Go West Young Man! Westward Expansion, 1840-19...",[-0.2710217 -1.2888566 -0.5610585 ... 0.75...


In [54]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

In [9]:
qdrantClient = QdrantClient(url=QDRANT_CONFIG["QDRANT_URL"],
            port=QDRANT_CONFIG["QDRANT_PORT"],
            grpc_port=6334,
            verify=False,
            timeout=100, prefer_grpc=True)

In [18]:
qdrantClient.create_collection(
        collection_name="HistoryBook2",
        vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
    )

True

In [20]:
df['embeddings'].to_list()

['[-0.75826144 -0.5804633  -1.4329212  ...  0.8780683  -0.33990186\n -1.7377317 ]',
 '[-0.73815656 -0.7213167  -1.4992899  ...  0.65346897 -0.8199464\n -1.0350413 ]',
 '[-0.485235   -0.8373444  -1.5882397  ...  0.59700125 -0.6756595\n -1.9720663 ]',
 '[-0.71489537 -0.57621986 -1.3793063  ...  0.5432745  -0.73588616\n -1.7768742 ]',
 '[-0.49304003 -0.5477071  -0.84136444 ...  0.41015854 -1.1017628\n -2.750595  ]',
 '[-0.6606756  -0.18924186 -0.955282   ...  0.20554122 -0.80455923\n -1.5927515 ]',
 '[-0.68428683  0.12068403 -1.0230826  ...  0.402972   -0.8604808\n -2.5965726 ]',
 '[-0.30776384 -0.24421668 -0.9967042  ...  1.0829319  -0.7310315\n -1.3606317 ]',
 '[-1.305133  -0.7248721 -1.4861192 ...  0.2549205 -1.3249207 -0.8769289]',
 '[-1.0422764   0.30438748 -0.6421106  ...  0.75522697 -0.6593059\n -1.7595869 ]',
 '[-0.5190124  -0.5726031  -1.0878228  ...  0.39448604 -0.49061322\n -1.810447  ]',
 '[-0.9634311  -0.44658753 -1.0343702  ...  0.61864233 -0.48537672\n -1.962707  ]',
 '[-0.

In [11]:
import uuid

In [12]:
uuid.uuid4()

UUID('b5c8c21a-979d-4ff5-b730-9b1a09c8a555')

In [13]:
df = df.rename(columns = {
    "CHAPTER":"payload.chapter",
    "TEXT":"payload.summary"
})

In [14]:
df

,payload.summary,payload.chapter,combined_text,embeddings
0,The election of PresidentFranklin Delano Roose...,"Franklin Roosevelt and the New Deal, 1932-1941","Franklin Roosevelt and the New Deal, 1932-1941...",[-0.75826144 -0.5804633 -1.4329212 ... 0.87...
1,"Roosevelt understood the need to ""save the pat...","Franklin Roosevelt and the New Deal, 1932-1941","Franklin Roosevelt and the New Deal, 1932-1941...",[-0.73815656 -0.7213167 -1.4992899 ... 0.65...
2,Franklin Roosevelt was part of the political e...,"Franklin Roosevelt and the New Deal, 1932-1941","Franklin Roosevelt and the New Deal, 1932-1941...",[-0.485235 -0.8373444 -1.5882397 ... 0.59...
3,As Hoover grew more morose and physically unwe...,"Franklin Roosevelt and the New Deal, 1932-1941","Franklin Roosevelt and the New Deal, 1932-1941...",[-0.71489537 -0.57621986 -1.3793063 ... 0.54...
4,"By the 1932 presidential election, Hoover's po...","Franklin Roosevelt and the New Deal, 1932-1941","Franklin Roosevelt and the New Deal, 1932-1941...",[-0.49304003 -0.5477071 -0.84136444 ... 0.41...
...,...,...,...,...
2012,"In their place came big business, with the inf...","Go West Young Man! Westward Expansion, 1840-1900","Go West Young Man! Westward Expansion, 1840-19...",[-0.09806645 -0.6857236 -0.5511301 ... 0.50...
2013,Settlers encroaching on Native American land c...,"Go West Young Man! Westward Expansion, 1840-1900","Go West Young Man! Westward Expansion, 1840-19...",[-0.36336237 -1.075941 0.50755715 ... 0.35...
2014,Although the Americanization policy formulated...,"Go West Young Man! Westward Expansion, 1840-1900","Go West Young Man! Westward Expansion, 1840-19...",[-0.63523734 -0.7510578 0.2216021 ... -0.10...
2015,"In the nineteenth century, the Hispanic, Chine...","Go West Young Man! Westward Expansion, 1840-1900","Go West Young Man! Westward Expansion, 1840-19...",[-0.2710217 -1.2888566 -0.5610585 ... 0.75...


In [42]:
for idx,row in df.iterrows():
    print(row['embeddings_new'])
    print(type(row['embeddings_new']))

[-0.75826144 -0.5804633  -1.4329212  ...  0.8780683  -0.33990186
 -1.7377317 ]
<class 'numpy.ndarray'>
[-0.73815656 -0.7213167  -1.4992899  ...  0.65346897 -0.8199464
 -1.0350413 ]
<class 'numpy.ndarray'>
[-0.485235   -0.8373444  -1.5882397  ...  0.59700125 -0.6756595
 -1.9720663 ]
<class 'numpy.ndarray'>
[-0.71489537 -0.57621986 -1.3793063  ...  0.5432745  -0.73588616
 -1.7768742 ]
<class 'numpy.ndarray'>
[-0.49304003 -0.5477071  -0.84136444 ...  0.41015854 -1.1017628
 -2.750595  ]
<class 'numpy.ndarray'>
[-0.6606756  -0.18924186 -0.955282   ...  0.20554122 -0.80455923
 -1.5927515 ]
<class 'numpy.ndarray'>
[-0.68428683  0.12068403 -1.0230826  ...  0.402972   -0.8604808
 -2.5965726 ]
<class 'numpy.ndarray'>
[-0.30776384 -0.24421668 -0.9967042  ...  1.0829319  -0.7310315
 -1.3606317 ]
<class 'numpy.ndarray'>
[-1.305133  -0.7248721 -1.4861192 ...  0.2549205 -1.3249207 -0.8769289]
<class 'numpy.ndarray'>
[-1.0422764   0.30438748 -0.6421106  ...  0.75522697 -0.6593059
 -1.7595869 ]
<class 

In [43]:
df

,TEXT,CHAPTER,combined_text,embeddings,embeddings_new
0,The election of PresidentFranklin Delano Roose...,"Franklin Roosevelt and the New Deal, 1932-1941","Franklin Roosevelt and the New Deal, 1932-1941...",[-0.75826144 -0.5804633 -1.4329212 ... 0.87...,"[-0.75826144, -0.5804633, -1.4329212, -0.31208..."
1,"Roosevelt understood the need to ""save the pat...","Franklin Roosevelt and the New Deal, 1932-1941","Franklin Roosevelt and the New Deal, 1932-1941...",[-0.73815656 -0.7213167 -1.4992899 ... 0.65...,"[-0.73815656, -0.7213167, -1.4992899, -0.45305..."
2,Franklin Roosevelt was part of the political e...,"Franklin Roosevelt and the New Deal, 1932-1941","Franklin Roosevelt and the New Deal, 1932-1941...",[-0.485235 -0.8373444 -1.5882397 ... 0.59...,"[-0.485235, -0.8373444, -1.5882397, -0.5109964..."
3,As Hoover grew more morose and physically unwe...,"Franklin Roosevelt and the New Deal, 1932-1941","Franklin Roosevelt and the New Deal, 1932-1941...",[-0.71489537 -0.57621986 -1.3793063 ... 0.54...,"[-0.71489537, -0.57621986, -1.3793063, -0.5976..."
4,"By the 1932 presidential election, Hoover's po...","Franklin Roosevelt and the New Deal, 1932-1941","Franklin Roosevelt and the New Deal, 1932-1941...",[-0.49304003 -0.5477071 -0.84136444 ... 0.41...,"[-0.49304003, -0.5477071, -0.84136444, -0.3989..."
...,...,...,...,...,...
2012,"In their place came big business, with the inf...","Go West Young Man! Westward Expansion, 1840-1900","Go West Young Man! Westward Expansion, 1840-19...",[-0.09806645 -0.6857236 -0.5511301 ... 0.50...,"[-0.09806645, -0.6857236, -0.5511301, -0.80030..."
2013,Settlers encroaching on Native American land c...,"Go West Young Man! Westward Expansion, 1840-1900","Go West Young Man! Westward Expansion, 1840-19...",[-0.36336237 -1.075941 0.50755715 ... 0.35...,"[-0.36336237, -1.075941, 0.50755715, 0.0766683..."
2014,Although the Americanization policy formulated...,"Go West Young Man! Westward Expansion, 1840-1900","Go West Young Man! Westward Expansion, 1840-19...",[-0.63523734 -0.7510578 0.2216021 ... -0.10...,"[-0.63523734, -0.7510578, 0.2216021, -0.522790..."
2015,"In the nineteenth century, the Hispanic, Chine...","Go West Young Man! Westward Expansion, 1840-1900","Go West Young Man! Westward Expansion, 1840-19...",[-0.2710217 -1.2888566 -0.5610585 ... 0.75...,"[-0.2710217, -1.2888566, -0.5610585, -0.445528..."


In [44]:
for idx,row in df.iterrows():
    # print(row['embeddings'])
    # print(type(row['embeddings']))
    pt = PointStruct(
        id = str(uuid.uuid4()),
        vector = row['embeddings_new'],
        payload = {
            "chapter": row["CHAPTER"],
            "summary": row["TEXT"]
        })
    
    cloud_client.upsert(collection_name="HistoryBook", wait=True, points = [pt])


In [2]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct
from qdrant_client.models import VectorParams,Distance

In [3]:
qdrant_url = "https://cefe850c-b463-45e8-b554-eb69961cd6c4.us-east4-0.gcp.cloud.qdrant.io:6333"
qdrant_key = "EAnJmEmGj3gXeztCX8cMH8JXzn5oemxowMUwHZg0wnvYS7GcBTS8bw"

cloud_client = QdrantClient(url = qdrant_url, api_key= qdrant_key, verify=False)
cloud_client.recreate_collection(collection_name="HistoryBook", vectors_config=VectorParams(size=1024,distance=Distance.COSINE,on_disk=True))

C:\Users\Karthik Arun\AppData\Local\Temp\ipykernel_3964\3795597355.py:5: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  cloud_client.recreate_collection(collection_name="HistoryBook", vectors_config=VectorParams(size=1024,distance=Distance.COSINE,on_disk=True))


True

In [ ]:
local_client = QdrantClient("localhost",port=6333)
local_client.migrate(cloud_client,[""])